In [0]:
DROP MATERIALIZED VIEW IF EXISTS mart.batsman_summary_mv;

CREATE MATERIALIZED VIEW mart.batsman_summary_mv AS
SELECT
    p.player_id,
    LOWER(p.full_name)                    AS batsman_name,
    t.team_name,
    ot.team_name       AS opponent_team,
    dm.match_id,
    dm.event_name,
    dm.match_type,
    dm.event_date,
    dm.season,

    -- Aggregates
    SUM(fd.runs_off_bat)                  AS runs_scored,
    SUM(fd.extras)                        AS extras,
    SUM(CASE WHEN fd.is_boundary_4 THEN 1 ELSE 0 END) AS fours,
    SUM(CASE WHEN fd.is_boundary_6 THEN 1 ELSE 0 END) AS sixes,
    COUNT(*)                              AS balls_faced,
    SUM(CASE WHEN fd.is_wicket THEN 1 ELSE 0 END)     AS times_out,

    -- Derived metrics
    ROUND(
        SUM(fd.runs_off_bat)::DECIMAL
        / NULLIF(SUM(CASE WHEN fd.is_wicket THEN 1 ELSE 0 END), 0),
        2
    ) AS batting_average,
    ROUND(
        (SUM(fd.runs_off_bat)::DECIMAL / NULLIF(COUNT(*), 0)) * 100,
        2
    ) AS strike_rate

FROM dw.fact_delivery fd
JOIN dw.dim_player p ON fd.striker_id = p.player_id
JOIN dw.dim_team   t ON p.team_id = t.team_id
JOIN dw.dim_match  dm ON fd.match_id = dm.match_id
JOIN dw.dim_team ot  ON fd.bowling_team_id = ot.team_id

GROUP BY
    p.player_id,
    p.full_name,
    t.team_name,
    ot.team_name,
    dm.match_id,
    dm.event_name,
    dm.match_type,
    dm.event_date,
    dm.season;

In [0]:
SELECT
*
FROM
mart.batsman_summary_mv
WHERE
batsman_name LIKE '%koh%'
ORDER BY match_id;

In [0]:
DROP MATERIALIZED VIEW IF EXISTS mart.batsman_career_mv;

CREATE MATERIALIZED VIEW mart.batsman_career_mv AS
SELECT
    p.player_id,
    LOWER(p.full_name) AS batsman_name,
    t.team_name,

    COUNT(DISTINCT dm.match_id)                         AS matches,
    SUM(fd.runs_off_bat)                                AS total_runs,
    SUM(CASE WHEN fd.is_boundary_4 THEN 1 ELSE 0 END)   AS total_fours,
    SUM(CASE WHEN fd.is_boundary_6 THEN 1 ELSE 0 END)   AS total_sixes,
    SUM(CASE WHEN fd.is_wicket THEN 1 ELSE 0 END)       AS times_out,

    ROUND(
        SUM(fd.runs_off_bat)::DECIMAL /
        NULLIF(SUM(CASE WHEN fd.is_wicket THEN 1 ELSE 0 END), 0),
        2
    ) AS batting_average,

    ROUND(
        (SUM(fd.runs_off_bat)::DECIMAL / NULLIF(COUNT(*), 0)) * 100,
        2
    ) AS strike_rate

FROM dw.fact_delivery fd
JOIN dw.dim_player p ON fd.striker_id = p.player_id
JOIN dw.dim_team   t ON p.team_id = t.team_id
JOIN dw.dim_match  dm ON fd.match_id = dm.match_id
GROUP BY p.player_id, p.full_name, t.team_name;

In [0]:
SELECT * FROM mart.batsman_career_mv WHERE batsman_name LIKE '%kohli%';

In [0]:
DROP MATERIALIZED VIEW IF EXISTS mart.batsman_dismissal_mv;

CREATE MATERIALIZED VIEW mart.batsman_dismissal_mv AS
SELECT
    p.player_id,
    LOWER(p.full_name) AS batsman_name,
    fd.wicket_kind,
    COUNT(*) AS dismissals
FROM dw.fact_delivery fd
JOIN dw.dim_player p ON fd.striker_id = p.player_id
WHERE fd.is_wicket = TRUE
GROUP BY p.player_id, p.full_name, fd.wicket_kind;

In [0]:
SELECT * FROM mart.batsman_dismissal_mv WHERE batsman_name LIKE '%kohli%';